In [11]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from pandas import DataFrame
from sklearn.neighbors import KNeighborsClassifier

In [12]:
def get_im_cv2(path):
    img = cv2.imread(path)
    height, width, channels = img.shape
    #img = cv2.normalize(img, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    img = cv2.resize(img, (32, 32))
    
    return img

In [13]:
def load_train():
    X_train = []
    X_train_id = []
    y_train = []
    start_time = time.time()

    print('Read train images')
    folders = ['Automobile',
               'Comics_and_Cartoons',
               'Famous_Personalities',
               'Festivals_and_Occasions',
               'Graffiti_and_Illustrations',
               'Movies_and_TV_shows',
               'Music',
               'Nature',
               'No_Theme',
               'Patterns_and_Ethnic',
               'Signs_and_Symbols',
               'Spiritual',
               'Sports',
               'Superheroes',
               'Typography',
               'Vintage'
              ]
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join('D:\Misc\CrowdAnalytix\Identifying Themes from Mobile Case Images\Data Raw\Train', fld, '*.jpg')
        files = glob.glob(path)
        for i in range(len(files)):
            flbase = os.path.basename(files[i])
            img = get_im_cv2(files[i])
            X_train.append(img)
            X_train_id.append(flbase)
            y_train.append(index)   #what does this do?
    print len(X_train)
    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, y_train, X_train_id

In [14]:
def load_test():
    path = os.path.join('D:\Misc\CrowdAnalytix\Identifying Themes from Mobile Case Images\Data Raw\Test', '*.jpg')
    files = sorted(glob.glob(path))

    X_test = []
    X_test_id = []
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl)
        X_test.append(img)
        X_test_id.append(flbase)

    return X_test, X_test_id

In [16]:
Xtrall, Ytrall, Itrall = load_train()
X_test, X_test_id = load_test()

Read train images
Load folder Automobile (Index: 0)
Load folder Comics_and_Cartoons (Index: 1)
Load folder Famous_Personalities (Index: 2)
Load folder Festivals_and_Occasions (Index: 3)
Load folder Graffiti_and_Illustrations (Index: 4)
Load folder Movies_and_TV_shows (Index: 5)
Load folder Music (Index: 6)
Load folder Nature (Index: 7)
Load folder No_Theme (Index: 8)
Load folder Patterns_and_Ethnic (Index: 9)
Load folder Signs_and_Symbols (Index: 10)
Load folder Spiritual (Index: 11)
Load folder Sports (Index: 12)
Load folder Superheroes (Index: 13)
Load folder Typography (Index: 14)
Load folder Vintage (Index: 15)
4592
Read train data time: 64.55 seconds


In [17]:
# flatten out all images to be one-dimensional
Xtrall = np.array(Xtrall, dtype=np.uint8)
Ytrall = np.array(Ytrall, dtype=np.uint8)

Xtrall = Xtrall.astype('float32')
Xtrall = Xtrall / 255

msk = np.random.rand(len(Xtr)) < 0.8
Xval = Xtrall[~msk]
Yval = Ytrall[~msk]
Ytr = Ytrall[msk]
Xtr = Xtrall[msk]

Xtr_rows = Xtr.reshape(Xtr.shape[0], 32 * 32 * 3) 
Xval_rows = Xval.reshape(Xval.shape[0], 32 * 32 * 3) 

X_test = np.array(X_test, dtype=np.uint8)

X_test = X_test.astype('float32')
X_test = X_test / 255
X_test_rows = X_test.reshape(X_test.shape[0], 32 * 32 * 3) 

In [18]:
# find hyperparameters that work best on the validation set
validation_accuracies = []
for k in [1, 3, 5, 10, 20, 50, 100]:
  
  # use a particular value of k and evaluation on validation data
  nn = KNeighborsClassifier(n_neighbors=k)
  nn.fit(Xtr_rows, Ytr)
  # here we assume a modified NearestNeighbor class that can take a k as input
  Yval_predict = nn.predict(Xval_rows)
  acc = np.mean(Yval_predict == Yval)
  print 'accuracy: %f' % (acc,)

  # keep track of what works on the validation set
  validation_accuracies.append((k, acc))

accuracy: 0.523272
accuracy: 0.521862
accuracy: 0.526093
accuracy: 0.531735
accuracy: 0.511989
accuracy: 0.478138
accuracy: 0.451340


In [20]:
validation_accuracies

[(1, 0.52327221438645977),
 (3, 0.52186177715091675),
 (5, 0.5260930888575458),
 (10, 0.53173483779971786),
 (20, 0.51198871650211564),
 (50, 0.47813822284908319),
 (100, 0.45133991537376589)]

In [21]:
nn = KNeighborsClassifier(n_neighbors=10)
nn.fit(Xtr_rows, Ytr)

X_test_predict = nn.predict(X_test_rows)
DataFrame(X_test_id,X_test_predict).to_csv("nn_submit_KnnL2.csv")